In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Notebooks taken for reference:
* https://www.kaggle.com/angqx95/data-science-workflow-top-2-with-tuning
* https://www.kaggle.com/cheesu/house-prices-1st-approach-to-data-science-process
* https://www.kaggle.com/datafan07/top-1-approach-eda-new-models-and-stacking#Model-Results
* https://www.kaggle.com/itslek/stack-blend-lrs-xgb-lgb-house-prices-k-v17

In [ ]:
#importing the required libraries for...
#handling and visualizing the data...
#and creating Machine Learning Models...

import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.linear_model import Ridge
from xgboost import XGBRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor

In [ ]:
#importing the data

train = pd.read_csv("/kaggle/input/home-data-for-ml-course/train.csv", index_col = 0, keep_default_na=False)
test = pd.read_csv("/kaggle/input/home-data-for-ml-course/test.csv", index_col = 0, keep_default_na=False)

Note: Pandas treats NA as NaN by default so explicitly it has been mentioned while reading the CSV file because some of our categorical features have NA as a separate class with meaning.

In [ ]:
#checking the data and format first-hand

print(train.shape)
train.head()

In [ ]:
#checking the data and format first-hand

print(test.shape)
test.head()

A quick analysis on a rough notebook made us realize that certain numerical features had missing values denoted by NA and hence now we need to make them disappear from training.<br>
<br>
Additionally certain categorical features that have NA as a separate class also had missing values represented by NA that now went unnoticed. So we find them explicity and make them disappear from both the sets.<br>
<br>
Apart from this certain categorical features that do not have NA as a separate class contain values that we will have to remove.<br>
<br>
Initially we will only remove the values from training set and then later we will work and process on test set.

In [ ]:
#LotFrontage column is supposed to be purely numerical but...
print(train.loc[:, ['LotFrontage']].info())
train.loc[train['LotFrontage']=='NA', ['LotFrontage']]

So LotFrontage apparently has 259 missing values.

In [ ]:
#another purely numerical feature that has NA is MasVnrArea
print(train.loc[:, ['MasVnrArea']].info())
train.loc[train['MasVnrArea']=='NA', ['MasVnrArea']]

MasVnrArea therefore has 8 missing values.

In [ ]:
#and the last numerical feature that has NA is GarageYrBlt
print(train.loc[:, ['GarageYrBlt']].info())
train.loc[train['GarageYrBlt']=='NA', ['GarageYrBlt']]

GarageYrBlt has apparently 81 missing values.

In [ ]:
#a categorical feature that cannot have NA value but possess in our dataset is Electrical
train.loc[train['Electrical']=='NA', ['Electrical']]

Electrical therefore has 1 missing value.

In [ ]:
#contardictory values in these two fields of MasVnrArea and MasVnrType
train.loc[((train['MasVnrArea']=='0') & (train['MasVnrType']!='None')) | ((train['MasVnrArea']!='0') & (train['MasVnrType']=='None')), ['MasVnrArea', 'MasVnrType']]

Here we can consider the MasVnrArea as missing somewhere and 0 somewhere...<br>
Missing Values in MasVnrArea(Id): 689, 1242<br>
Missing Values in MasVnrType(Id): 625, 1301, 1335<br>
0 values in MasVnrArea(Id): 774, 1231<br>

In [ ]:
#column of MasVnrType also has certain missing values because
train.loc[train['MasVnrType']=='NA',['MasVnrType']]

Oof...MasVnrType has 8 missing values.

In [ ]:
#one of the categorical feature that can have NA value but
#contradicts some other feature is BsmtExposure

train.loc[(train['TotalBsmtSF']>0) & (train['BsmtExposure']=='NA'), ['TotalBsmtSF', 'BsmtExposure']]


That's one missing value for sure.

In [ ]:
#similarly

train.loc[(train['TotalBsmtSF']>0) & (train['BsmtFinType2']=='NA'), ['TotalBsmtSF', 'BsmtFinType2']]

Another missing value. Duh!

I realized later that we should rather check for missing values in test set at this point only. We could have efficiently loop over with train and test set simultaneously but meh... 

In [ ]:
#numerical feature LotFrotage coming in first
print(test.loc[:, ['LotFrontage']].info())
test.loc[test['LotFrontage']=='NA', ['LotFrontage']]

227 missing rows! duh

In [ ]:
#another numerical feature
print(test.loc[:, ['MasVnrArea']].info())
test.loc[test['MasVnrArea']=='NA', ['MasVnrArea']]

15 missing values!

In [ ]:
#another one
print(test.loc[:, ['TotalBsmtSF']].info())
test.loc[test['TotalBsmtSF']=='NA', ['TotalBsmtSF']]

1 missing value

In [ ]:
#let's just club some features and display them
test.loc[:, ['BsmtUnfSF', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtFullBath', 'BsmtHalfBath']].info()
test.loc[(test['BsmtUnfSF']=='NA') | (test['BsmtFinSF1']=='NA') | (test['BsmtFinSF2']=='NA') | (test['BsmtFullBath']=='NA') |
         (test['HalfBath']=='NA'), ['BsmtUnfSF', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtFullBath', 'BsmtHalfBath']]

Missing and explicit 0s needed.

In [ ]:
#ah missing...missing...actually not missing but the area is supposed to be 0 based on other columns
print(test.loc[:, ['GarageArea']].info())
test.loc[test['GarageArea']=='NA', ['GarageArea']]

1 value needed.

In [ ]:
#GarageCars as well
print(test.loc[:, ['GarageCars']].info())
test.loc[test['GarageCars']=='NA', ['GarageCars']]

1 value needed.

In [ ]:
#and the last feature of numerical type is
print(test.loc[:, ['GarageYrBlt']].info())
test.loc[test['GarageYrBlt']=='NA', ['GarageYrBlt']]

This feature has 78 missing values. Turns out test set has more missing values than training set. lmao

In [ ]:
#MasVnrType doesn't have a class NA and still
print(test.loc[:, ['MasVnrType']].info())
test.loc[test['MasVnrType']=='NA', ['MasVnrType']]

16 missing values. 

In [ ]:
#Utilities doesn't have a class NA and still
print(test.loc[:, ['Utilities']].info())
test.loc[test['Utilities']=='NA', ['Utilities']]

2 missing values.

In [ ]:
#Utilities doesn't have a class NA and still
print(test.loc[:, ['MSZoning']].info())
test.loc[test['MSZoning']=='NA', ['MSZoning']]

In [ ]:
#contradictory columns values also need to be checked
test.loc[(test['TotalBsmtSF']!='0') & (test['BsmtExposure']=='NA'), ['TotalBsmtSF', 'BsmtExposure']]

aha! 2 values that contradict... and 1 that we need to figure out how to handle

In [ ]:
#contradictory columns values also need to be checked
test.loc[(test['TotalBsmtSF']!='0') & (test['BsmtQual']=='NA'), ['TotalBsmtSF', 'BsmtQual']]

2 more. eh this is boring thing to do.

In [ ]:
#contradictory columns values also need to be checked
test.loc[(test['TotalBsmtSF']!='0') & (test['BsmtCond']=='NA'), ['TotalBsmtSF', 'BsmtCond']]

3 more values!

this is a tricky one

In [ ]:
#contardictory values in these two fields of MasVnrArea and MasVnrType
test.loc[((test['MasVnrArea']=='0') & (test['MasVnrType']!='None')) | ((test['MasVnrArea']!='0') & (test['MasVnrType']=='None')), ['MasVnrArea', 'MasVnrType']]

Missing Value in MasVnrType(Id): 1670<br>
Missing Value in MasVnrArea(Id): 2320<br>
0 value in MasVnrArea(Id): 2453<br>

### Categorization of Features on basis of the type of the Variable they belong to

* Nominal Values: MSSubClass, MSZoning, LandContour, Utilities, LotConfig, Neighborhood, Condition1, Condition2, BldgType, HouseStyle, RoofStyle, RoofMatl, Exterior1st, Exterior2nd, MasVnrType, Foundation, Heating, CentralAir, GarageType, MiscFeature, MoSold, SaleType, SaleCondition
* Continuos: LotFrontage, LotArea, YearBuilt, YearRemodAdd, MasVnrArea, BsmtFinSF1, BsmtFinSF2, BsmtUnfSF, TotalBsmtSF, 1stFlrSF, 2ndFlrSF, LowQualFinSF, GrLivArea, GarageArea, GarageYrBlt, WoodDeckSF, OpenPorchSF, EnclosedPorch, 3SsnPorch, ScreenPorch, PoolArea, MiscVal
* Discrete: BsmtFullBath, BsmtHalfBath, FullBath, HalfBath, Bedroom, Kitchen, TotRmsAbvGrd, Fireplaces, GarageCars, YrSold 
* Ordinal: Street, Alley, LotShape, LandSlope, OverallQual, OverallCond, ExterQual, ExterCond, BsmtQual, BsmtCond, BsmtExposure, BsmtFinType1, BsmtFinType2, HeatingQC, Electrical, KitchenQual, Functional, FireplaceQu, GarageFinish, GarageQual, GarageCond, PavedDrive, PoolQC, Fence

Before we begin with discriptive analysis let us just remove the NA values that are actually missing in the dataset.

In [ ]:
#handling the numerical columns first
for dataset in [train, test]:
    dataset.loc[:, ['LotFrontage', 'MasVnrArea', 'GarageYrBlt']] = dataset.loc[:, ['LotFrontage', 'MasVnrArea', 'GarageYrBlt']].replace(to_replace ="NA", 
                     value = np.nan)
    dataset.loc[:, ['LotFrontage', 'MasVnrArea', 'GarageYrBlt']] = dataset.loc[:, ['LotFrontage', 'MasVnrArea', 'GarageYrBlt']].astype(float)
    print('*'*50)
    print(dataset.loc[:, ['LotFrontage', 'MasVnrArea', 'GarageYrBlt']].info())

In [ ]:
#let us now explicitly remove the NA value from Electrical
for dataset in [train, test]:
    dataset.loc[dataset['Electrical']=='NA', 'Electrical'] = np.nan
    print('*'*50)
    print(dataset.loc[:, ['Electrical']].info())

In [ ]:
#now let's remove or edit values from MasVnrArea and MasVnrType
train.loc[[689, 1242], 'MasVnrArea'] = np.nan
test.loc[[2320], 'MasVnrArea'] = np.nan
train.loc[[625, 1301, 1335], 'MasVnrType'] = np.nan
test.loc[[1670], 'MasVnrType'] = np.nan
train.loc[[774, 1231], 'MasVnrArea'] = 0
test.loc[[2453], 'MasVnrArea'] = 0
train.loc[:, ['MasVnrArea']] = train.loc[:, ['MasVnrArea']].astype(float)
test.loc[:, ['MasVnrArea']] = test.loc[:, ['MasVnrArea']].astype(float)
print(train.loc[:, ['MasVnrArea', 'MasVnrType']].info())
print('*'*50)
print(test.loc[:, ['MasVnrArea', 'MasVnrType']].info())

In [ ]:
#now let's remove NA values from MasVnrType
for dataset in [train, test]:
    dataset.loc[:, ['MasVnrType']] = dataset.loc[:, ['MasVnrType']].replace(to_replace ="NA", value = np.nan)
    print('*'*50)
    print(dataset.loc[:, ['MasVnrType']].info())

In [ ]:
#removing NA values from Utilities
test.loc[[1916, 1946], 'Utilities'] = np.nan

In [ ]:
#removing NA values from MSZoning
test.loc[[1916, 2217, 2251, 2905], ['MSZoning']] = np.nan

In [ ]:
#BsmtExposure here we come
train.loc[949, 'BsmtExposure'] = np.nan
test.loc[[1488, 2349], 'BsmtExposure'] = np.nan
print(train.loc[:, ['BsmtExposure']].info())
print('*'*50)
print(test.loc[:, ['BsmtExposure']].info())

In [ ]:
#Bsmt related data in test
test.loc[2121, ['BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF']] = [0, 0, 0, 0]
test.loc[:, ['BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF']] = test.loc[:, ['BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF']].astype(float)
test.loc[2121, ['BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF']]

In [ ]:
#Bsmt related data in test
test.loc[[2121, 2189], ['BsmtFullBath', 'BsmtHalfBath']] = [[0, 0], [0, 0]]
test.loc[:, ['BsmtFullBath', 'BsmtHalfBath']] = test.loc[:, ['BsmtFullBath', 'BsmtHalfBath']].astype(float)
test.loc[[2121, 2189], ['BsmtFullBath', 'BsmtHalfBath']]

In [ ]:
#Bsmt related data in test
test.loc[[2218, 2219], ['BsmtQual']] = [[np.nan], [np.nan]]
test.loc[:, ['BsmtQual']].info()

In [ ]:
#Bsmt related data in test
test.loc[[2041, 2186, 2525], ['BsmtCond']] = [[np.nan], [np.nan], [np.nan]]
test.loc[:, ['BsmtCond']].info()

In [ ]:
#0s in Garage related data
test.loc[2577, ['GarageArea', 'GarageCars']] = [0, 0]
test.loc[:, ['GarageArea', 'GarageCars']] = test.loc[:, ['GarageArea', 'GarageCars']].astype(float)
test.loc[2577, ['GarageArea', 'GarageCars']]

In [ ]:
#BsmtFinType2 and then we are done
train.loc[333, 'BsmtFinType2'] = np.nan
train.loc[:, ['BsmtFinType2']].info()

Now we can peacefully begin with analysis of other data!<br>
Statistically starting would be a good thing!

In [ ]:
#statistical description
print(train.describe().shape)
train.describe()

* In total 37 columns have numerical values including Id and few others which need certain transformation.
* Some columns might have outliers that need to be checked.
* Few columns have missing values as we have already talked about that.
* The last entry of a house being built or remodelled was recorded in the year 2010.
* Overall condition and quality on average seem to be around 5-6.
* A lot of houses do not have fancy or redundant factors like having unnecessary amount of baths or a wood deck. We might want to remove such features that mostly have a single value or class.

In [ ]:
#description of categorical or other attributes
print(train.describe(include='O').shape)
train.describe(include='O')

* There are 43 columns with non-numerical data. Some of these might have ordinal values. We need to check.
* Few columns have missing values but the count is less. Let's see what we can do about that.
* Most of the houses have same classfier values.
* As the numerical ordinal ratings for house and other features were on average average, same is the case here.

In [ ]:
#finding the insights into missing data
print(train.info())
print('*'*50)
print(test.info())

In [ ]:
#getting the count idea of missing values in train set
missing = {}
for i in range(0, len(train.columns)):
    if train.loc[:, [train.columns[i]]].isnull().sum()[train.columns[i]]>0:
        missing[train.columns[i]] = train.loc[:, [train.columns[i]]].isnull().sum()[train.columns[i]]
missing = pd.DataFrame([[i, j] for i, j in zip(list(missing.keys()), list(missing.values()))], columns=['Column', '#missing'])
missing

In [ ]:
#getting the count idea of missing values in test set
missing = {}
for i in range(0, len(test.columns)):
    if test.loc[:, [test.columns[i]]].isnull().sum()[test.columns[i]]>0:
        missing[test.columns[i]] = test.loc[:, [test.columns[i]]].isnull().sum()[test.columns[i]]
missing = pd.DataFrame([[i, j] for i, j in zip(list(missing.keys()), list(missing.values()))], columns=['Column', '#missing'])
missing

Some columns with very less missing values can be dropped or replaced with the most frequent value(categorical and discrete).<br>
We can also think for other techniques. For example houses in the same locality or having similar features can be used to find missing values. In fact that is the gist in KNN imputing technqiue. NeIgHbOrS!
Some other techniques can also be used which we can discuss later.

Now we aim to analyze the data even more. We will use viusalization tools to our aid.<br>
We shall first separate out the columns containing numerical values and those containing objects (classes of some kind).

In [ ]:
#creating separate numerical and non-numerical dataframe for analysis
#a quick analysis in data description tells us that MSSubClass is actually a categorical column with
#numerical values
numerical = train.select_dtypes(exclude=['object']).drop(['MSSubClass'], axis=1).copy()
numerical.columns

In [ ]:
#creating separate numerical and non-numerical dataframe for analysis
categorical = train.select_dtypes(include=['object']).copy()
categorical['MSSubClass'] = train['MSSubClass']
categorical.columns

First we will perform Univariate analysis by checking the distribution and various other plots.

In [ ]:
#let us first check the skewness of the target variable

#graph = sns.FacetGrid(train, height=4, aspect=1.33)
#graph.map(sns.distplot, 'SalePrice')
fig, axes = plt.subplots(figsize=(12, 6))
sns.distplot(train['SalePrice'])
print(train['SalePrice'].skew())

Obviously the data is skewed with most of the data lying towards the left region where SalePrice is between 100000 and 300000. The target variable is positive skewed. A good thing to do would be to convert it to a Normal distribution

In [ ]:
#we apply log function to the target variable

#graph = sns.FacetGrid(train, height=4, aspect=1.33)
#graph.map(sns.distplot, 'SalePrice')
fig, axes = plt.subplots(figsize=(12, 6))
sns.distplot(np.log(train['SalePrice']))
print(np.log(train['SalePrice']).skew())

That looks like a nice distribution. Fairly symmetric. GG<br>
We need to log-transform the target variable in the preprocessing stage.

In [ ]:
#let us check if the independent variables also are skewed or...
#if they need any transformation

fig, ax = plt.subplots(figsize=(12, 18))
for i in range(0, len(numerical.columns)):
    fig.add_subplot(9,4,i+1)
    plt.title(str(numerical.iloc[:, i].skew()))
    sns.distplot(numerical.iloc[:, i], kde=False)
fig.tight_layout(pad=1.0)

* Some features have discrete values.
* The ones with continuous values are skewed.
* Many features have one value (0) that over-dominates the other values.

In [ ]:
#let us go for statistical inference
#boxplots will help us for the same and help in detecting the presence of outliers if any
#it will be better if we only plot for continuos variables
cont = ['LotFrontage', 'LotArea', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 
        'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'GarageArea',
        'GarageYrBlt', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal']

fig, ax = plt.subplots(figsize=(12, 18))
for i in range(0, len(cont)):
    fig.add_subplot(6,4,i+1)
    sns.boxplot(y = cont[i], data = numerical)
fig.tight_layout(pad=1.0)

Because some continuous variables have the same value (mostly 0), the box plot will tend to find outliers.<br>
Other than that some other features also seem to contain certain outliers.<br>
We might have to decide a threshold for each of them.

In [ ]:
#now let's get a count idea of categorical ones visually

fig, ax = plt.subplots(figsize=(12,20))
for i in range(len(categorical.columns)):
    fig.add_subplot(9,5,i+1)
    sns.countplot(x=categorical.iloc[:,i], data=categorical.dropna())
    plt.xticks(rotation=90)
fig.tight_layout(pad=1.0)

In some of the features we see that one of the class over-dominates the other classes by a large factor.<br>
Gotta check that later.

Now to get an idea of relationships between features with themselves and target variable let us begin with Bivariate analysis.<br>
We begin with numerical features first.

In [ ]:
#first let's get the idea of correlation between numerical features

fig, ax = plt.subplots(figsize=(12, 8))
correlation = numerical.corr()
sns.heatmap(correlation, mask=correlation<0.8, linewidth=0.5, cmap='Blues')

Clearly some of these features are strongly related and we surely do not want redundancy.
There are four pairs of such strong correlation that we seek to remove later.
These are:
* 1stFlrSF and TotalBsmtSF
* GarageYrBlt and YearBuilt
* TotRmsAbvGrd and GrLivArea
* GarageCars and GarageArea

In [ ]:
#let us now get an idea of how other numeric features contribute in predicting the SalePrice
#by the measure of their correlation with it

correlation = numerical.corr()
correlation[['SalePrice']].sort_values(['SalePrice'],ascending=False)

Obviously it is evident that some numerical features can help us in predicting the SalePrice strongly.<br>
People indeed are concerned with the OverallQual, GrLivArea, GarageArea and many other features that are shown in the above DataFrame.

In [ ]:
#lastly let us just do a scatterplot analysis to visualize the linear relationship
#between numerical and target feature

fig, ax = plt.subplots(figsize=(12, 18))
for i in range(0, len(numerical.columns)):
    fig.add_subplot(9,4,i+1)
    sns.scatterplot(x = numerical.columns[i], y = 'SalePrice', data=numerical)
fig.tight_layout(pad=1.0)

There's certainly some linearity, but not that much that it would be worth finding a quantitative measure for them.<br>
<br>
Let us now move towards preprocessing the data from our finding.<br>
We shall be performing:
* Finding missing values
* Removing outliers
* Encoding the features correctly for both nominal and ordinal categories
* Creating bands/bins of data if necessary
* OneHotEncoding or creating the dummy columns for nominal features
* Removing redundant columns
* Generating new columns if necessary
<br><br>
*let's go* **pubgnoises**

In [ ]:
#first we remove the redundant numerical features that we found have high correlation
#we shall remove the one from the pair that has comparitvely less correlation with the
#target variable
train = train.drop(['1stFlrSF', 'GarageYrBlt', 'TotRmsAbvGrd', 'GarageArea'], axis=1)
test = test.drop(['1stFlrSF', 'GarageYrBlt', 'TotRmsAbvGrd', 'GarageArea'], axis=1)    
print(train.info())
print('*'*50)
print(test.info())

In [ ]:
#removing categorical columns with a lot of same values (96%)
cat_col = train.select_dtypes(include=['object']).columns
overfit_cat = []
for i in cat_col:
    counts = train[i].value_counts()
    zeros = counts.iloc[0]
    if zeros / len(train[i]) * 100 > 96:
        overfit_cat.append(i)
print(overfit_cat)

In [ ]:
#removing numericalcal columns with a lot of same values (96%)
num_col = train.select_dtypes(exclude=['object']).columns
overfit_num = []
for i in num_col:
    counts = train[i].value_counts()
    zeros = counts.iloc[0]
    if zeros / len(train[i]) * 100 > 96:
        overfit_num.append(i)
print(overfit_num)

In [ ]:
train = train.drop(overfit_cat, axis=1)
train = train.drop(overfit_num, axis=1)
test = test.drop(overfit_cat, axis=1)
test = test.drop(overfit_num, axis=1)
print(train.info())
print('*'*50)
print(test.info())

In [ ]:
#removing outliers
train = train.drop(train[train['LotArea'] > 100000].index)
train.shape

In [ ]:
#locating missing values in MasVnrArea and MasVnrType
print('Train Set')
print(train.loc[train['MasVnrType'].isnull(), ['MasVnrArea', 'MasVnrType']])
print('Test Set')
print(test.loc[test['MasVnrType'].isnull(), ['MasVnrArea', 'MasVnrType']])

In [ ]:
#now we replace the missing values in MasVnrType with BrkFace
#for those which have certain MasVnrArea associated with them
train.loc[[625, 1301, 1335], ['MasVnrType']] = ['BrkFace' for _ in range(0, 3)]
test.loc[[1670, 2611], ['MasVnrType']] = ['BrkFace', 'BrkFace']

In [ ]:
#replacing MasVnrArea with 0 and MasVnrType with None
#when none of the information is provided about them together
train.loc[train['MasVnrType'].isnull(), ['MasVnrArea', 'MasVnrType']] = [[0, 'None']]*len(train.loc[train['MasVnrType'].isnull(), ['MasVnrArea', 'MasVnrType']])
test.loc[test['MasVnrType'].isnull(), ['MasVnrArea', 'MasVnrType']] = [[0, 'None']]*len(test.loc[test['MasVnrType'].isnull(), ['MasVnrArea', 'MasVnrType']])

In [ ]:
#replacing BsmtExposure with No
train.loc[train['BsmtExposure'].isnull(), ['BsmtExposure']] = ['No']*len(train.loc[train['BsmtExposure'].isnull(), ['BsmtExposure']])
test.loc[test['BsmtExposure'].isnull(), ['BsmtExposure']] = ['No']*len(test.loc[test['BsmtExposure'].isnull(), ['BsmtExposure']])

In [ ]:
#replacing BsmtFinType2 with Rec
train.loc[train['BsmtFinType2'].isnull(), ['BsmtFinType2']] = ['Rec']*len(train.loc[train['BsmtFinType2'].isnull(), ['BsmtFinType2']])
train.loc[train['BsmtFinType2'].isnull(), ['BsmtFinType2']]

In [ ]:
#replacing BsmtQual, BsmtCond with TA
test.loc[test['BsmtQual'].isnull(), ['BsmtQual']] = ['TA']*len(test.loc[test['BsmtQual'].isnull(), ['BsmtQual']])
test.loc[test['BsmtCond'].isnull(), ['BsmtCond']] = ['TA']*len(test.loc[test['BsmtCond'].isnull(), ['BsmtCond']])

In [ ]:
#replacing Electrical with SBrkr
train.loc[train['Electrical'].isnull(), ['Electrical']] = ['SBrkr']

In [ ]:
#replacing MSZoning with RL
test.loc[test['MSZoning'].isnull(), ['MSZoning']] = ['RL']*len(test.loc[test['MSZoning'].isnull(), ['MSZoning']])

In [ ]:
#creating a merged dataset to make our job of encoding and finidng missing values easy
merged_df = pd.concat([train.iloc[:, :-1], test], axis=0)
merged_df.shape

In [ ]:
#creating dummy variables
mssubclass_dummies = pd.get_dummies(merged_df.MSSubClass, prefix="MSSubClass")
mszoning_dummies = pd.get_dummies(merged_df.MSZoning, prefix="MSZoning")
merged_df.drop(['MSSubClass', 'MSZoning'], axis = 1, inplace = True)
merged_df = pd.concat([mssubclass_dummies.iloc[:, 1:], mszoning_dummies.iloc[:, 1:], merged_df], axis = 1)

In [ ]:
#creating dummy variables
landcontour_dummies = pd.get_dummies(merged_df.LandContour, prefix="LandContour")
lotconfig_dummies = pd.get_dummies(merged_df.LotConfig, prefix="LotConfig")
merged_df = pd.concat([merged_df.loc[:, :'LandContour'], landcontour_dummies.iloc[:, 1:],
                       merged_df.loc[:, ['LandContour']], lotconfig_dummies.iloc[:, 1:], merged_df.loc[:, 'LotConfig':]], axis = 1)
merged_df.drop(['LandContour', 'LotConfig'], axis = 1, inplace = True)

In [ ]:
#creating dummy variables
neighborhood_dummies = pd.get_dummies(merged_df.Neighborhood, prefix="Neighborhood")
condition1_dummies = pd.get_dummies(merged_df.Condition1, prefix="Condition1")
bldgtype_dummies = pd.get_dummies(merged_df.BldgType, prefix="BldgType")
housestyle_dummies = pd.get_dummies(merged_df.HouseStyle, prefix="HouseStyle")
merged_df = pd.concat([merged_df.loc[:, :'Neighborhood'], neighborhood_dummies.iloc[:, 1:],
                       merged_df.loc[:, ['Neighborhood']], condition1_dummies.iloc[:, 1:],
                       merged_df.loc[:, ['Condition1']], bldgtype_dummies.iloc[:, 1:],
                       merged_df.loc[:, ['BldgType']], housestyle_dummies.iloc[:, 1:],
                       merged_df.loc[:, 'HouseStyle':]], axis=1)
merged_df.drop(['Neighborhood', 'Condition1', 'BldgType', 'HouseStyle'], axis = 1, inplace = True)

In [ ]:
#creating dummy variables
roofstyle_dummies = pd.get_dummies(merged_df.RoofStyle, prefix="RoofStyle")
exterior1st_dummies = pd.get_dummies(merged_df.Exterior1st, prefix="Exterior1st")
exterior2nd_dummies = pd.get_dummies(merged_df.Exterior2nd, prefix="Exterior2nd")
masvnrtype_dummies = pd.get_dummies(merged_df.MasVnrType, prefix="MasVnrType")
merged_df = pd.concat([merged_df.loc[:, :'RoofStyle'], roofstyle_dummies.iloc[:, 1:],
                       merged_df.loc[:, ['RoofStyle']], exterior1st_dummies.iloc[:, 1:],
                       merged_df.loc[:, ['Exterior1st']], exterior2nd_dummies.iloc[:, 1:],
                       merged_df.loc[:, ['Exterior2nd']], masvnrtype_dummies.iloc[:, 1:],
                       merged_df.loc[:, 'MasVnrType':]], axis=1)
merged_df.drop(['RoofStyle', 'Exterior1st', 'Exterior2nd', 'MasVnrType'], axis = 1, inplace = True)

In [ ]:
#creating dummy variables
foundation_dummies = pd.get_dummies(merged_df.Foundation, prefix="Foundation")
merged_df = pd.concat([merged_df.loc[:, :'Foundation'], foundation_dummies.iloc[:, 1:],
                       merged_df.loc[:, 'Foundation':]], axis=1)
merged_df.drop(['Foundation'], axis = 1, inplace = True)

In [ ]:
#creating dummy variables
centralair_dummies = pd.get_dummies(merged_df.CentralAir, prefix="CentralAir")
merged_df = pd.concat([merged_df.loc[:, :'CentralAir'], centralair_dummies.iloc[:, 1:],
                       merged_df.loc[:, 'CentralAir':]], axis=1)
merged_df.drop(['CentralAir'], axis = 1, inplace = True)

In [ ]:
#creating dummy variables
garagetype_dummies = pd.get_dummies(merged_df.GarageType, prefix="GarageType")
merged_df = pd.concat([merged_df.loc[:, :'GarageType'], garagetype_dummies.iloc[:, 1:],
                       merged_df.loc[:, 'GarageType':]], axis=1)
merged_df.drop(['GarageType'], axis = 1, inplace = True)

In [ ]:
#creating dummy variables
mosold_dummies = pd.get_dummies(merged_df.MoSold, prefix="MoSold")
merged_df = pd.concat([merged_df.loc[:, :'MoSold'], mosold_dummies.iloc[:, 1:],
                       merged_df.loc[:, 'MoSold':]], axis=1)
merged_df.drop(['MoSold'], axis = 1, inplace = True)

In [ ]:
#creating dummy variables
saletype_dummies = pd.get_dummies(merged_df.SaleType, prefix="SaleType")
salecondition_dummies = pd.get_dummies(merged_df.SaleCondition, prefix="SaleCondition")
merged_df = pd.concat([merged_df.loc[:, :'SaleType'], saletype_dummies.iloc[:, 1:],
                       merged_df.loc[:, ['SaleType']], salecondition_dummies.iloc[:, 1:],
                       merged_df.loc[:, 'SaleCondition':]], axis=1)
merged_df.drop(['SaleType', 'SaleCondition'], axis = 1, inplace = True)

In [ ]:
#mapping ordinal values
ordinal_map = {'Alley' : {'Grvl':2, 'Pave':1, 'NA':0},
'LotShape' : {'Reg':3, 'IR1':2, 'IR2':1, 'IR3':0},
'LandSlope' : {'Gtl':2, 'Mod':1, 'Sev':0},
'ExterQual' : {'Ex':4, 'Gd':3, 'TA':2, 'Fa':1, 'Po':0},
'ExterCond' : {'Ex':4, 'Gd':3, 'TA':2, 'Fa':1, 'Po':0},
'BsmtQual' : {'Ex':5, 'Gd':4, 'TA':3, 'Fa':2, 'Po':1, 'NA':0},
'BsmtCond' : {'Ex':5, 'Gd':4, 'TA':3, 'Fa':2, 'Po':1, 'NA':0},
'BsmtExposure' : {'Gd':4, 'Av':3, 'Mn':2, 'No':1, 'NA':0},
'BsmtFinType1' : {'GLQ':6, 'ALQ':5, 'BLQ':4, 'Rec':3, 'LwQ':2, 'Unf':1, 'NA':0},
'BsmtFinType2' : {'GLQ':6, 'ALQ':5, 'BLQ':4, 'Rec':3, 'LwQ':2, 'Unf':1, 'NA':0},
'HeatingQC' : {'Ex':4, 'Gd':3, 'TA':2, 'Fa':1, 'Po':0},
'Electrical' : {'SBrkr':4, 'FuseA':3, 'FuseF':2, 'FuseP':1, 'Mix':0},
'KitchenQual' : {'Ex':4, 'Gd':3, 'TA':2, 'Fa':1, 'Po':0},
'Functional' : {'Typ':7, 'Min1':6, 'Min2':5, 'Mod':4, 'Maj1':3, 'Maj2':2, 'Sev':1, 'Sal':0},
'FireplaceQu' : {'Ex':5, 'Gd':4, 'TA':3, 'Fa':2, 'Po':1, 'NA':0},
'GarageFinish' : {'Fin':3, 'RFn':2, 'Unf':1, 'NA':0},
'GarageQual' : {'Ex':5, 'Gd':4, 'TA':3, 'Fa':2, 'Po':1, 'NA':0},
'GarageCond' : {'Ex':5, 'Gd':4, 'TA':3, 'Fa':2, 'Po':1, 'NA':0},
'PavedDrive' : {'Y':2, 'P':1, 'N':0},
'Fence' : {'GdPrv':4, 'MnPrv':3, 'GdWo':2, 'MnWw':1, 'NA':0}}

In [ ]:
#mapping ordinal values
for col in list(ordinal_map.keys()):
    merged_df[col] = merged_df[col].map(ordinal_map[col])

In [ ]:
#separating the datasets again for ease at performing next steps
train_X = merged_df.loc[1:1460, :].copy()
train_y = train.iloc[:, -1].copy()
test_X = merged_df.loc[1461:, :].copy()

In [ ]:
#computing missing values through KNN technique
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=5)
imputer.fit(train_X.iloc[:, :])
train_X.iloc[:, :]=imputer.transform(train_X.iloc[:, :])
test_X.iloc[:, :]=imputer.transform(test_X.iloc[:, :])

In [ ]:
#converting dtype for appropriate columns
cols = ['LotFrontage', 'LotArea', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '2ndFlrSF', 
        'LowQualFinSF', 'GrLivArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', 'ScreenPorch']
for i in train_X.columns:
    if i not in cols:
        train_X.loc[:, [i]] = train_X.loc[:, [i]].astype(int)
        test_X.loc[:, [i]] = test_X.loc[:, [i]].astype(int)

In [ ]:
#applying log transformation to skewed variables
skewed_cols = ['LotFrontage', 'LotArea', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'TotalBsmtSF',
              'GrLivArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', 'ScreenPorch']
for dataset in [train_X, test_X]:
    for col in skewed_cols:
        dataset[col] = np.log1p(dataset[col])
train_y = np.log(train_y)

In [ ]:
#creating training and validation sets
from sklearn.model_selection import train_test_split
train_X = train_X.values
train_y = train_y.values
test_X = test_X.values
X_train, X_val, y_train, y_val = train_test_split(train_X, train_y, test_size=0.2, random_state=13)

In [ ]:
#creating copies of the training and validation sets
#and making them scaled for some other regression models
from sklearn.preprocessing import StandardScaler
X_train_scaled, X_val_scaled, y_train_scaled, y_val_scaled = X_train.copy(), X_val.copy(), y_train.copy(), y_val.copy()
test_X_scaled = test_X.copy()
sc_X = StandardScaler()
sc_X.fit(X_train_scaled)
X_train_scaled = sc_X.transform(X_train_scaled)
X_val_scaled = sc_X.transform(X_val_scaled)
test_X_scaled = sc_X.transform(test_X_scaled)

In [ ]:
#defining helper functions
from sklearn.model_selection import cross_val_score
def trainer(model, X, y, test):
    model.fit(X, y)
    return model.predict(test), cross_val_score(estimator=model, X=X, y=y, cv=10, scoring = 'neg_mean_absolute_error')

In [ ]:
#Base Models
#Ridge Regressor
rid_reg = Ridge(random_state=13)
rid_y, rid_score = trainer(rid_reg, X_train_scaled, y_train_scaled, X_val_scaled)

#Support Vector Regressor
svr_reg = SVR()
svr_y, svr_score = trainer(svr_reg, X_train_scaled, y_train_scaled, X_val_scaled)

#Random Forest Regressor
rf_reg = RandomForestRegressor(random_state=13)
rf_y, rf_score = trainer(rf_reg, X_train, y_train, X_val)

#XGBoost Regressor
xgb_reg = XGBRegressor()
xgb_y, xgb_score = trainer(xgb_reg, X_train, y_train, X_val)

In [ ]:
#comparing the score returned by cross_val_score
#even though the scoring is done on log values of predictions
scores = {'Model':['Ridge', 'SVR', 'Random Forest', 'XGBoost'], 'MAE': [rid_score.mean(), svr_score.mean(), rf_score.mean(), xgb_score.mean()],
          'SD': [rid_score.std(), svr_score.std(), rf_score.std(), xgb_score.std()]}
scores = pd.DataFrame([[i, -1*j, k] for i, j, k in zip(scores['Model'], scores['MAE'], scores['SD'])], columns=list(scores.keys()))
scores

In [ ]:
#now we inverse log the predictions returned to check MAE
y_val = np.exp(y_val)
y_val_scaled = np.exp(y_val_scaled)
rid_y = np.exp(rid_y)
svr_y = np.exp(svr_y)
rf_y = np.exp(rf_y)
xgb_y = np.exp(xgb_y)

In [ ]:
#checking the performance of our models based on MAE
from sklearn.metrics import mean_absolute_error
rid_val_score = mean_absolute_error(y_val_scaled, rid_y)
svr_val_score = mean_absolute_error(y_val_scaled, svr_y)
rf_val_score = mean_absolute_error(y_val, rf_y)
xgb_val_score = mean_absolute_error(y_val, xgb_y)
val_scores = pd.DataFrame([['Ridge', rid_val_score],
                           ['SVR', svr_val_score],
                           ['Random Forest', rf_val_score],
                           ['XGBoost', xgb_val_score]], columns=['Model', 'MAE'])
val_scores

In [ ]:
#now we blend the models and give vote weight
#on basis of their MAE on validation set
blend_y = (0.35*rid_y) + (0.1*svr_y) + (0.2*rf_y) + (0.35*xgb_y)
blend_score = mean_absolute_error(y_val, blend_y)
blend_score = pd.DataFrame([['Blend Model', blend_score]], columns = ['Model', 'MAE'])
blend_score

In [ ]:
#predicting the test set now
#Base Models
#Ridge Regressor
rid_reg = Ridge(random_state=13)
rid_y, rid_score = trainer(rid_reg, X_train_scaled, y_train_scaled, test_X_scaled)

#Support Vector Regressor
svr_reg = SVR()
svr_y, svr_score = trainer(svr_reg, X_train_scaled, y_train_scaled, test_X_scaled)

#Random Forest Regressor
rf_reg = RandomForestRegressor(random_state=13)
rf_y, rf_score = trainer(rf_reg, X_train, y_train, test_X)

#XGBoost Regressor
xgb_reg = XGBRegressor()
xgb_y, xgb_score = trainer(xgb_reg, X_train, y_train, test_X)

In [ ]:
#now we inverse log the predictions and blend them
rid_y = np.exp(rid_y)
svr_y = np.exp(svr_y)
rf_y = np.exp(rf_y)
xgb_y = np.exp(xgb_y)
blend_y = (0.35*rid_y) + (0.1*svr_y) + (0.2*rf_y) + (0.35*xgb_y)

In [ ]:
#creating the submission file
submission = pd.DataFrame({'Id':pd.read_csv("/kaggle/input/home-data-for-ml-course/test.csv", keep_default_na=False)['Id'], 'SalePrice':blend_y})
submission.head()

In [ ]:
submission.to_csv("Housing_Submission.csv", index = False)